<a href="https://colab.research.google.com/github/ehho34/PyTorch/blob/main/03_%EC%84%A0%ED%98%95%ED%9A%8C%EA%B7%80(Linear_Regression).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**03.선형회귀(Linear Regression)**

##**01-선형 회귀**

###**데이터에 대한 이해(Data Definition)**

In [ ]:
"""
-예측을 위해 사용하는 데이터 == 훈련 데이터셋(training dataset)
-학습이 끝난 후에 이 모델이 얼마나 잘 작동하는지 판별 == 테스트 데이터셋(test dataset)

-훈련 데이터셋의 구성
  -모델을 학습시키기 위한 데이터는 파이토치의 텐서 형태여야 함!
  -입력과 출력은 서로 다른 텐서에 저장해야 함!(입력은 x, 출력은 y)<-보편적으로~
  EX) 공부시간과 점수(아래 사진 참고)
  x_train = torch.FloatTensor([[1],[2],[3]])
  y_train = torch.FloatTensor([[2],[4],[6]])

-가설(Hypothesis) 수립
  -머신 러닝에서 식을 세울 때 이 식을 가설이라고 함

-선형 회귀
 -이 가설은 널리 알려져 있으니까 고민 불필요
 -선형 회귀란? : 학습 데이터와 가장 잘 맞는 하나의 직선을 찾는 일
                 일반적으로 이런 식     y = Wx + b 
                                     H(x) = Wx + b 을 가진다 (가설의 H를 따서 y대신 쓴 것뿐임)
                                     -여기서 x와 곱해지는 W를 가중치(Weight)라고 하며, b를 편향(bias)라고 함
                                     -W와 b는 중학교 수학 과정 직선의 방정식에서 기울기와 y절편에 해당
"""

'\n-예측을 위해 사용하는 데이터 == 훈련 데이터셋(training dataset)\n-학습이 끝난 후에 이 모델이 얼마나 잘 작동하는지 판별 == 테스트 데이터셋(test dataset)\n\n-훈련 데이터셋의 구성\n  -모델을 학습시키기 위한 데이터는 파이토치의 텐서 형태여야 함!\n  -입력과 출력은 서로 다른 텐서에 저장해야 함!(입력은 x, 출력은 y)<-보편적으로~\n  EX) 공부시간과 점수(아래 사진 참고)\n  x_train = torch.FloatTensor([[1],[2],[3]])\n  y_train = torch.FloatTensor([[2],[4],[6]])\n\n-가설(Hypothesis) 수립\n  -머신 러닝에서 식을 세울 때 이 식을 가설이라고 함\n\n-선형 회귀\n -이 가설은 널리 알려져 있으니까 고민 불필요\n -선형 회귀란? : 학습 데이터와 가장 잘 맞는 하나의 직선을 찾는 일\n                 일반적으로 이런 식     y = Wx + b \n                                     H(x) = Wx + b 을 가진다 (가설의 H를 따서 y대신 쓴 것뿐임)\n                                     -여기서 x와 곱해지는 W를 가중치(Weight)라고 하며, b를 편향(bias)라고 함\n                                     -W와 b는 중학교 수학 과정 직선의 방정식에서 기울기와 y절편에 해당\n'

In [ ]:
"""
-비용 함수(Cost Function)에 대한 이해
=손실 함수(loss function)=오차 함수(error function)=목적 함수(objective function)
★ 특히 비용 함수, 손실 함수 기억해두기 ★

-(실제값-예측값)의 제곱을 다 더하고 데이터의 개수로 나눈 것! -> 평균 제곱 오차(Mean Squared Error, MSE)
-Cost(W,b)를 최소로 만드는 W와 b를 구하면 훈련 데이터를 가장 잘 나타내는 직선을 구할 수 있다!
"""

'\n-비용 함수(Cost Function)에 대한 이해\n=손실 함수(loss function)=오차 함수(error function)=목적 함수(objective function)\n★ 특히 비용 함수, 손실 함수 기억해두기 ★\n\n-(실제값-예측값)의 제곱을 다 더하고 데이터의 개수로 나눈 것! -> 평균 제곱 오차(Mean Squared Error, MSE)\n-Cost(W,b)를 최소로 만드는 W와 b를 구하면 훈련 데이터를 가장 잘 나타내는 직선을 구할 수 있다!\n'

In [ ]:
"""
-옵티마이저 - 경사 하강법(Gradient Descent)
-기울기가 지나치게 크면 실제값과 예측값의 오차가 커기고 기울기가 지나치게 작아도 마찬가지
-사실 b 또한 위와 같음

-cost가 최소화 되는 지점은 접선의 기울기가 0이 되는 지점 = 미분값이 0이 되는 지점
"""

'\n-옵티마이저 - 경사 하강법(Gradient Descent)\n-기울기가 지나치게 크면 실제값과 예측값의 오차가 커기고 기울기가 지나치게 작아도 마찬가지\n-사실 b 또한 위와 같음\n\n-cost가 최소화 되는 지점은 접선의 기울기가 0이 되는 지점 = 미분값이 0이 되는 지점\n'

###**파이토치로 선형 회귀 구현하기**

####**1.기본 셋팅**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
#현재 실습하고 있는 코드를 다시 실행해도 다음에도 같은 결과가 나올 수 있게 랜덤 시들들 줄이기
torch.manual_seed(1)

####**2.변수 선언**

In [ ]:
x_train = torch.FloatTensor([[1],[2],[3]])
y_train = torch.FloatTensor([[2],[4],[6]])

In [ ]:
print(x_train)
print(x_train.shape)

tensor([[1.],
        [2.],
        [3.]])
torch.Size([3, 1])


In [ ]:
print(y_train)
print(y_train.shape)

tensor([[2.],
        [4.],
        [6.]])
torch.Size([3, 1])


####**3.가중치와 편향의 기초화**

In [ ]:
#가중치 W를 0으로 초기화하고 학습을 통해 값이 변경되는 변수임을 명시함.
W = torch.zeros(1, requires_grad=True)  #requires_grad=True <- '이 변수는 학습을 통해 계속 값이 변경되는 변수임'
#가중치 W를 출력
print(W)

tensor([0.], requires_grad=True)


In [ ]:
#편향 b를 0으로 초기화하고 얘도 학습을 통해 값이 변경되는 변수임을 명시하자!
b = torch.zeros(1, requires_grad=True)
print(b)

tensor([0.], requires_grad=True)


In [ ]:
#W와 b의 값이 모두 0이기 때문에 현재 이 직선의 방정식은 y = 0 X x + 0 --> x에 어떤 수가 들어가더라도 가설은 0을 예측할 것임!

####**4.가설 세우기**
H(x) = Wx + b

In [ ]:
hypothesis = x_train * W + b
print(hypothesis)

tensor([[0.],
        [0.],
        [0.]], grad_fn=<AddBackward0>)


####**5.비용 함수 선언하기**

In [ ]:
#평균 구하기(torch.mean 사용)
cost = torch.mean((hypothesis - y_train) ** 2)
print(cost)

tensor(18.6667, grad_fn=<MeanBackward0>)


####**6.경사 하강법 구현하기**


In [ ]:
optimizer = optim.SGD([W,b],lr=0.01)  #SDG(경사 하강법의 일종), lr(learning rate, 학습률)

In [ ]:
#미분을 통해 얻은 기울기를 0으로 초기화(기울기를 초기화해야지 새로운 가중치 편향에 대해서 새로운 기울기를 구할 수 있다.)
#gradient를 0으로 초기화
optimizer.zero_grad()
#비용 함수를 미분하여 gradient 계산 
cost.backward() #이 함수를 호출하면 인수로 들어갔던 W와 b에서 리턴되는 변수들의 기울기에 학습률(learning rate) 0.01을 곱하여 빼줌으로서 업데이트

####**7.전체 코드**

In [ ]:
#데이터
x_train = torch.FloatTensor([[1],[2],[3]])
y_train = torch.FloatTensor([[2],[4],[6]])

#모델 초기화
W = torch.zeros(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)

#optimizer 설정
optimizer = optim.SGD([W,b], lr=0.01)

nb_epochs = 1999 #원하는만큼 경사 하강법을 반복
for epoch in range(nb_epochs + 1):

  #H(x) 계산
  hypothesis = x_train * W + b

  #cost 계산
  cost = torch.mean((hypothesis - y_train) ** 2)

  #cost로 H(x) 개선
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  #100번마다 로그 출력
  if epoch % 100 ==0:
    print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
        epoch, nb_epochs, W.item(), b.item(), cost.item()
,     ))

Epoch    0/1999 W: 0.187, b: 0.080 Cost: 18.666666
Epoch  100/1999 W: 1.746, b: 0.578 Cost: 0.048171
Epoch  200/1999 W: 1.800, b: 0.454 Cost: 0.029767
Epoch  300/1999 W: 1.843, b: 0.357 Cost: 0.018394
Epoch  400/1999 W: 1.876, b: 0.281 Cost: 0.011366
Epoch  500/1999 W: 1.903, b: 0.221 Cost: 0.007024
Epoch  600/1999 W: 1.924, b: 0.174 Cost: 0.004340
Epoch  700/1999 W: 1.940, b: 0.136 Cost: 0.002682
Epoch  800/1999 W: 1.953, b: 0.107 Cost: 0.001657
Epoch  900/1999 W: 1.963, b: 0.084 Cost: 0.001024
Epoch 1000/1999 W: 1.971, b: 0.066 Cost: 0.000633
Epoch 1100/1999 W: 1.977, b: 0.052 Cost: 0.000391
Epoch 1200/1999 W: 1.982, b: 0.041 Cost: 0.000242
Epoch 1300/1999 W: 1.986, b: 0.032 Cost: 0.000149
Epoch 1400/1999 W: 1.989, b: 0.025 Cost: 0.000092
Epoch 1500/1999 W: 1.991, b: 0.020 Cost: 0.000057
Epoch 1600/1999 W: 1.993, b: 0.016 Cost: 0.000035
Epoch 1700/1999 W: 1.995, b: 0.012 Cost: 0.000022
Epoch 1800/1999 W: 1.996, b: 0.010 Cost: 0.000013
Epoch 1900/1999 W: 1.997, b: 0.008 Cost: 0.000008

###**optimizer.zero_grad()가 필요한 이유**

In [ ]:
#파이토치는 미분을 통해 얻은 기울기를 이전에 계산된 기울기 값에 누적시키는 특징이 있습니다.

In [ ]:
import torch
w = torch.tensor(2.0, requires_grad=True)

nb_epochs = 20
for epoch in range(nb_epochs + 1):

  z =2*w

  z.backward()
  print('수식을 w로 미분한 값 : {}'.format(w.grad))

수식을 w로 미분한 값 : 2.0
수식을 w로 미분한 값 : 4.0
수식을 w로 미분한 값 : 6.0
수식을 w로 미분한 값 : 8.0
수식을 w로 미분한 값 : 10.0
수식을 w로 미분한 값 : 12.0
수식을 w로 미분한 값 : 14.0
수식을 w로 미분한 값 : 16.0
수식을 w로 미분한 값 : 18.0
수식을 w로 미분한 값 : 20.0
수식을 w로 미분한 값 : 22.0
수식을 w로 미분한 값 : 24.0
수식을 w로 미분한 값 : 26.0
수식을 w로 미분한 값 : 28.0
수식을 w로 미분한 값 : 30.0
수식을 w로 미분한 값 : 32.0
수식을 w로 미분한 값 : 34.0
수식을 w로 미분한 값 : 36.0
수식을 w로 미분한 값 : 38.0
수식을 w로 미분한 값 : 40.0
수식을 w로 미분한 값 : 42.0


###**torch.manual_seed()를 하는 이유**

In [1]:
#torch.manual_seed()를 사용한 프로그램의 결과는 다른 컴퓨터에서 실행시켜도 동일한 결과를 낸다.
#torch.manual_seed()는 난수 발생 순서와 값을 동일하게 보장해주는 특징을 갖고 있기 때문이다!

In [2]:
import torch

In [3]:
torch.manual_seed(3) #랜덤시드가 3
print('랜덤 시드가 3일 때')
for i in range(1,3):
  print(torch.rand(1))
#랜덤시드가 3일 때 난수 2개를 발생시켰더니 0.0043과 0.1056이 나옴  

랜덤 시드가 3일 때
tensor([0.0043])
tensor([0.1056])


In [4]:
torch.manual_seed(5)
print('랜덤 시드가 5일 때')
for i in range(1,3):
  print(torch.rand(1))

랜덤 시드가 5일 때
tensor([0.8303])
tensor([0.1261])


In [5]:
#다시 랜덤 시드값을 처음에 한 것처럼 3으로 돌려보자! 그럼 어떻게 될까?
torch.manual_seed(3)
print('랜덤 시드가 3일 때')
for i in range(1,3):
  print(torch.rand(1))
  #동일한 값 도출

랜덤 시드가 3일 때
tensor([0.0043])
tensor([0.1056])


In [6]:
#tersor에는 requires_grab라는 속성이 있는데, requires+grab=True로 설정하면 자동 미분 기능이 적용된다.
#이렇게 설정된 텐서에서 연산을 하면 계산 그래프가 생성됨. backward 함수를 호출하면 그래프로부터 자동으로 미분이 계산된다! 